In [28]:
import pandas as pd
import requests
import altair as alt
import os
import functools
import numpy as np
from groq import Groq
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.chat_message_histories import StreamlitChatMessageHistory

In [38]:
!pip install plotly agent agent-executor

ERROR: Could not find a version that satisfies the requirement agent-executor (from versions: none)
ERROR: No matching distribution found for agent-executor


In [29]:
def fetch_sectors_dashboard_data(ticker, sectors_api_key):
    """
    Fetches key info, metrics, and price history from the LIVE Sectors.app API.
    """
    print(f"Fetching live data for {ticker} from Sectors.app...")
    headers = {"Authorization": f"Bearer {sectors_api_key}"}

    try:
        # 1. Fetch Company Info
        info_url = f"https://api.sectors.app/v1/company/{ticker}/profile"
        info_response = requests.get(info_url, headers=headers)
        info_response.raise_for_status()  # Raise error for bad responses
        info_data = info_response.json().get('data', {})

        # 2. Fetch Price History (1 year) - CORRECTED
        price_url = f"https://api.sectors.app/v1/company/{ticker}/charts?period=1y"
        price_response = requests.get(price_url, headers=headers)
        price_response.raise_for_status()
        price_data = price_response.json().get('data', [])

        if not price_data:
            print(f"No price data returned for {ticker}.")
            return None, None

        hist = pd.DataFrame(price_data)
        hist['Date'] = pd.to_datetime(hist['date'])
        hist['Close'] = hist['close'].astype(float)

        # 3. Fetch Key Metrics
        metrics_url = f"https://api.sectors.app/v1/company/{ticker}/metrics"
        metrics_response = requests.get(metrics_url, headers=headers)
        metrics_response.raise_for_status()
        metrics_data = metrics_response.json().get('data', {}).get('metrics', {})

        # --- Format Data for Dashboard ---
        hist['ma_50'] = hist['Close'].rolling(window=20).mean()

        last_price = hist['Close'].iloc[-1]
        prev_close = hist['Close'].iloc[-2] if len(hist) > 1 else last_price

        dashboard_data = {
            "ticker": ticker,
            "companyName": info_data.get("name", f"Data for {ticker}"),
            "logo_url": info_data.get("logo", "https://placehold.co/100x100/1a1c24/FFFFFF?text=LOGO&font=inter"),
            "lastPrice": last_price,
            "prevClose": prev_close,
            "peRatio": metrics_data.get("pe_ratio", "N/A"),
            "marketCap": metrics_data.get("market_cap", "N/A"),
            "priceHistory": hist.dropna()
        }

        print(f"Successfully fetched live data for {ticker}.")
        return dashboard_data, info_data  # Return info_data for the agent

    except requests.exceptions.HTTPError as http_err:
        print(
            f"HTTP error fetching data for {ticker}: {http_err}. Check API key and ticker.")
    except Exception as e:
        print(f"Error fetching live data for {ticker}: {e}")

    return None, None

In [ ]:
class SectorsInput(BaseModel):
    ticker: str = Field(description="The stock ticker, e.g., 'BMRI.JK'")


def _fetch_sectors_fundamentals(ticker, api_key):
    """
    A tool to get fundamental data for a stock from Sectors.app.
    This function accepts an API key.
    """
    headers = {"Authorization": f"Bearer {api_key}"}
    try:
        # Fetch quarterly statements
        url = f"https://api.sectors.app/v1/financials/quarterly/{ticker}/"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        statements = response.json().get('data', [])

        # Fetch profile for summary
        profile_url = f"https://api.sectors.app/v1/company/{ticker}/profile"
        profile_response = requests.get(profile_url, headers=headers)
        profile_response.raise_for_status()
        profile = profile_response.json().get('data', {})

        fundamentals = {
            "businessSummary": profile.get('description', 'No summary available.'),
            "sector": profile.get('sector', 'N/A'),
            "industry": profile.get('industry', 'N/A'),
            "latestQuarterlyStatement": statements[0] if statements else "No quarterly statement found."
        }
        return f"Fundamental data for {ticker} (from Sectors.app):\n\n{fundamentals}"
    except Exception as e:
        return f"Error fetching fundamentals: {e}"


def _fetch_sectors_news(ticker, api_key):
    """
    A tool to get the latest news articles for a stock from Sectors.app.
    This function accepts an API key.
    """
    headers = {"Authorization": f"Bearer {api_key}"}
    try:
        url = f"https://api.sectors.app/v1/company/{ticker}/news?limit=5"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        articles = response.json().get('data', [])

        if not articles:
            return f"No recent news found for {ticker}."

        news_formatted = [
            f"Title: {article['title']}\nSource: {article.get('source', 'N/A')}\nLink: {article['url']}\nPublished: {article['published_at']}"
            for article in articles
        ]
        return f"Latest news for {ticker} (from Sectors.app):\n\n" + "\n\n".join(news_formatted)
    except Exception as e:
        return f"Error fetching news: {e}"

In [31]:
def create_price_chart(price_history_df):
    """Renders an interactive price chart using Altair."""

    # Base chart
    base = alt.Chart(price_history_df).encode(
        x=alt.X('Date', title='Date'),
        tooltip=[
            alt.Tooltip('Date', format="%Y-%m-%d"),
            alt.Tooltip('Close', title='Close Price'),
            alt.Tooltip('ma_50', title='50-Day MA')
        ]
    ).interactive()

    # Price line
    price_line = base.mark_line(color='#4CAF50').encode(
        y=alt.Y('Close', title='Close Price (Rp)', scale=alt.Scale(zero=False))
    )

    # Moving average line
    ma_line = base.mark_line(color='#FFA500', strokeDash=[5, 5]).encode(
        y=alt.Y('ma_50', title='50-Day MA')
    )

    chart = price_line + ma_line
    return chart

In [32]:
# --- This cell is for testing in a notebook ---
from dotenv import load_dotenv

# Load variables from .env file if it exists
env_loaded = load_dotenv()
if env_loaded:
    print("Loaded environment variables from .env file.")
else:
    print("No .env file found. Relying on manually set environment variables.")

# 1. SET YOUR API KEYS (as environment variables or in a .env file)
# Example .env file content:
# GROQ_API_KEY="gsk_..."
# SECTORS_API_KEY="sk_..."

MY_GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
MY_SECTORS_API_KEY = os.environ.get("SECTORS_API_KEY")
TICKER = "BMRI.JK"

if not MY_GROQ_API_KEY or not MY_SECTORS_API_KEY:
    print("WARNING: GROQ_API_KEY or SECTORS_API_KEY not set. Please create a .env file or set them manually to run the notebook test.")
else:
    print(f"API Keys loaded. Ticker set to: {TICKER}")

Loaded environment variables from .env file.
API Keys loaded. Ticker set to: BMRI.JK


In [33]:
if MY_GROQ_API_KEY and MY_SECTORS_API_KEY:
    dashboard_data, stock_info = fetch_sectors_dashboard_data(
        TICKER, MY_SECTORS_API_KEY)
    if dashboard_data:
        print("--- Dashboard Data OK ---")
        print(f"Company: {dashboard_data['companyName']}")
        print(f"Last Price: {dashboard_data['lastPrice']}")
    else:
        print("Failed to fetch dashboard data. Check API key or ticker.")
else:
    print("Skipping test: API Keys not set.")

Fetching live data for BMRI.JK from Sectors.app...
HTTP error fetching data for BMRI.JK: 401 Client Error: Unauthorized for url: https://api.sectors.app/v1/company/BMRI.JK/profile/. Check API key and ticker.
Failed to fetch dashboard data. Check API key or ticker.


In [34]:
if 'dashboard_data' in locals() and dashboard_data:
    chart = create_price_chart(dashboard_data['priceHistory'])
    print("\n--- Chart OK ---")
    # In a notebook, display(chart) would render the interactive chart
    # display(chart)
    print("Altair chart object created successfully.")
else:
    print("Skipping chart: 'dashboard_data' not available.")

Skipping chart: 'dashboard_data' not available.


In [35]:
if MY_GROQ_API_KEY and MY_SECTORS_API_KEY:
    llm = ChatGroq(model="llama3-70b-8192",
                   api_key=MY_GROQ_API_KEY, temperature=0)

    fetch_fundamentals_with_key = functools.partial(
        _fetch_sectors_fundamentals, api_key=MY_SECTORS_API_KEY)
    fetch_news_with_key = functools.partial(
        _fetch_sectors_news, api_key=MY_SECTORS_API_KEY)

    tools = [
        Tool(name="getStockFundamentals", func=fetch_fundamentals_with_key,
             description="Fetches fundamental data for a given stock ticker.", args_schema=SectorsInput),
        Tool(name="getLatestNews", func=fetch_news_with_key,
             description="Fetches the latest news headlines for a given stock ticker.", args_schema=SectorsInput)
    ]

    prompt = hub.pull("hwchase17/react-chat")
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

    print("\n--- Agent OK ---")
    print(f"Agent and tools initialized for {TICKER}.")
else:
    print("Skipping agent setup: API Keys not set.")

ValidationError: 1 validation error for ChatGroq
__root__
  Client.__init__() got an unexpected keyword argument 'proxies' (type=type_error)

In [ ]:
if 'agent_executor' in locals():
    chat_history = [
        SystemMessage(
            content=f"You are a helpful analyst for {TICKER}. All data comes from Sectors.app."),
        AIMessage(content="Hello! How can I help?")
    ]

    user_query = "What is the latest news?"
    print(f"\n--- Testing Agent with query: '{user_query}' ---")

    response = agent_executor.invoke({
        "input": user_query,
        "chat_history": chat_history
    })

    print("\n--- Agent Response ---")
    print(response["output"])
else:
    print("Skipping agent test: Agent not initialized.")

Skipping agent test: Agent not initialized.
